In [1]:
"""
@authors: faurand, chardes, ehagensieker
"""
import tensorflow_datasets as tfds
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import math
import datetime
import tqdm
import pprint

# in a notebook, load the tensorboard extension, not needed for scripts
%load_ext tensorboard

In [7]:
train_ds, test_ds = tfds.load('mnist', split=['train', 'test'], as_supervised=True)

In [78]:
def new_target_fnc(ds, sequence_len):

  l = list()
  for i, elem in enumerate(ds):
    if (i % sequence_len) == 0:
      l.append(int(elem[1]))
    else:
      if (i % 2) == 0:
        l.append(int(l[i-1] + elem[1]))
      else:
        l.append(int(l[i-1] - elem[1]))
  return l

In [80]:
def prepare_data(data,batch_size = 32, seq_length = 4, augmentation = None): 
    '''
    prepare the dataset to have one-hot-vectors and values between -1 and 1
    '''
    #train_ds, test_ds = tfds.load('mnist', split=['train', 'test'], as_supervised=True)

     #flatten the images into one dimensional vector
    data = data.map(lambda img, target: (img[0], target))
    data = data.map(lambda img, target: (tf.reshape(img, (-1,)), target))
    #convert data from uint8 to float32
    data = data.map(lambda img, target: (tf.cast(img, tf.float32), tf.cast(target,tf.int32)))
    #normalization to have input of range [-1,1]
    data = data.map(lambda img, target: ((img/128.)-1., target))



    data = data.batch(seq_length).cache().shuffle(1028).batch(batch_size).prefetch(tf.data.AUTOTUNE)
        
    return data 

In [81]:
train_targets = tf.data.Dataset.from_tensor_slices(new_target_fnc(train_ds, 4))
test_targets = tf.data.Dataset.from_tensor_slices(new_target_fnc(test_ds, 4))

#Zip up datasets and new targets
train_ds = tf.data.Dataset.zip((train_ds, train_targets))
test_ds = tf.data.Dataset.zip((test_ds, test_targets))

#Prepare the new datasets
train = prepare_data(train_ds)
test = prepare_data(test_ds)

In [ ]:
for e in train.take(1):
  #print(train_ds.take(1))
  print(e[1])